# MNIST 데이터

In [ ]:
# 데이터 로드
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from scipy import io
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#서버 오류 -> 별도로 다운
#mnist = fetch_mldata("MNIST original")
#X = mnist.data / 255.0
#y = mnist.target

#7만개의 작은 숫자 이미지
#행 열이 반대로 되어있음 -> 전치
mnist = io.loadmat('mnist-original.mat') 
X = mnist['data'].T
y = mnist['label'].T

# grayscale 28x28 pixel = 784 feature
# 각 picel은 0~255의 값
# label = 1~10

print (X.shape, y.shape)

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df.head()

In [ ]:
df['y'] = y
print('Size of the dataframe: {}'.format(df.shape))

# 데이터 형태 시각화

In [ ]:
import matplotlib.pyplot as plt

rndperm = np.random.permutation(df.shape[0])

# Plot the graph
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,15):
    ax = fig.add_subplot(3,5,i+1, title="Digit: {}".format(str(df.loc[rndperm[i],'y'])) )
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))
plt.show()

## Mnist데이터를사용(8:2 비율로train set, test set split)

In [ ]:
#10000개만
X= df.loc[rndperm[:10000],:] 
y = X.pop('y')

In [ ]:
#train / test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## 원본데이터 & PCA 축소데이터 & LDA 축소데이터 비교
### 시각화 하기위해 2개의 feature로 차원축소 해보겠다.

In [ ]:
# 1. 원본데이터
pd.DataFrame(X_train).head() #784개의 feature로 나타내진것을 알 수 있다.

In [ ]:
from sklearn.decomposition import PCA
# test set을 transform 할때는 train의 fit으로 !
pca = PCA(n_components=2) # n_components는 feature의 개수이다.

%time pca.fit(X_train)

%time X_train_pca = pd.DataFrame(pca.transform(X_train))
%time X_test_pca = pd.DataFrame(pca.transform(X_test))

In [ ]:
X_train_pca.head()

In [ ]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [ ]:
y_train.reset_index(inplace=True)
del y_train['index']
y_test.reset_index(inplace=True)
del y_test['index']

In [ ]:
print(X_train_pca.shape, 
y_train.shape)

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('First and Second Principal Components colored by digit', fontsize = 20)

for i in range(10):
    ax.scatter(X_train_pca[0][y_train['y']==i]
               , X_train_pca[1][y_train['y']==i]
               , s = 10)

ax.legend(range(10))
ax.grid()# 


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=2)

%time lda.fit(X_train, y_train) # lda는 supervised learning으로 가장 분별정보가 높은 축을 찾아 차원축소 한다.

%time X_train_lda = pd.DataFrame(lda.transform(X_train))
%time X_test_lda = pd.DataFrame(lda.transform(X_test))

In [ ]:
X_train_lda.head()

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('LDA 1', fontsize = 15)
ax.set_ylabel('LDA 2', fontsize = 15)
ax.set_title('First and Second LDA colored by digit', fontsize = 20)

for i in range(10):
    ax.scatter(X_train_lda[0][y_train['y']==i]
            ,X_train_lda[1][y_train['y']==i]
            , s = 10)

ax.legend(range(10))
ax.grid()

# 지금까지배웠던다항분류기2개이상사용(KNN, random forest, NB 등등)
## -timestamp 찍어서training시간과testaccuracy비교하기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10) # 숫자가 10개니까 10개로 분류해본다.

%time knn.fit(X_train, y_train) 
print("ORIGINAL KNN SCORE:", knn.score(X_test, y_test)) 

%time knn.fit(X_train_pca, y_train) 
print("PCA KNN SCORE:", knn.score(X_test_pca, y_test)) 

%time knn.fit(X_train_lda, y_train) 
print("LDA KNN SCORE:", knn.score(X_test_lda, y_test)) 

'''
차원축소 안할경우 784개의 feature에 대해 거리를 전부 계산하기 때문에 시간이 30배이상으로 오래걸린다.
PCA를 2차원으로 했을경우 0.4365
LDA를 2차원으로 했을경우 0.5175 LDA가 조금 더 좋은이유는 지도학습 때문이라고 생각한다.
'''

#https://scikit-learn.org/stable/modules/neighbors.html

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

%time clf.fit(X_train, y_train)  
print("ORIGINAL RF SCORE:", clf.score(X_test, y_test)) 

%time clf.fit(X_train_pca, y_train)  
print("ORIGINAL RF SCORE:", clf.score(X_test_pca, y_test)) 

%time clf.fit(X_train_lda, y_train)  
print("ORIGINAL RF SCORE:", clf.score(X_test_lda, y_test)) 

# 출처: https://excelsior-cjh.tistory.com/166 [EXCELSIOR]

In [ ]:
'''
시간이 부족하여 다양한 하이퍼파라미터 튜닝이나 모델을 사용하지 못하고,
차원축소를 다양한 개수로 해보지 못한 점이 아쉽습니다..... 다음엔 더 열심히 해보겠습니다..
확실히 깨달은건 고차원의 데이터들을 다룰 수 있는 또 한가지의 방법을 배웠고
고유값 분해, lda와 pca의 차이점등은 확실히 알았습니다! 
'''